# Association Rule Learning

This project aim is find a relation between the products and suggest right product combination to the buyers bu using online retail II dataset.

We are going to follow the content below in this project:


1. Importing Libraries & Data 
2. Data Preprocessing
3. Preparation of Association Rules
4. Suggestion a product to the buyers. 

## Import Libraries and Upload Dataset

In association rule project, we're going to use apriori algorithm which is an algorithm that allows the implementation of Association Rules to detect products in the same basket. 

In [ ]:
import pandas as pd
import os
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
from mlxtend.frequent_patterns import apriori, association_rules

We create a function for excel datasets. We use google colab in the project, before we upload the dataset need a quick confuguration for drive usage.

In [ ]:
def load_dataset(data):
    path = os.getcwd()
    sheet = input("Which sheet you want to open?")
    return pd.read_excel(path + '/' + data + ".xlsx", sheet_name = sheet)
  # sheet name is "Year 2010-2011"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We change the directory to data's directory get from automaticly.

In [ ]:
os.chdir('/content/drive/MyDrive/Python/VBO/')

Excel file has two sheet for the years that we choose the second sheet "Year 2010-2011"

In [ ]:
df_ = load_dataset('online_retail_II')
df_.head()

Which sheet you want to open?Year 2010-2011


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
df = df_.copy()


## Data Preprocessing


In [ ]:
def summary_data(df):
    print("  Head   :")
    print(df.head())
    print("  Shape   :")
    print(df.shape)
    print("  Describe   :")
    print(df.quantile([0, 0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.95, 0.99, 1]).T)
    print("  Null   :")
    print(df.isnull().sum())
    print("  Columns   :")
    print(df.columns)
    print("  Duplicate   :")
    print(df.duplicated().sum())
    print("  Index max   :")
    print(df.index.max())
    print("  Info   :")
    print(df.info())


In [ ]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [ ]:
def retail_data_prep(dataframe):
    dataframe.dropna(inplace=True)
    # dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[~dataframe["Invoice"].str.contains(r"[a-zA-Z]+", na=False)]
    dataframe = dataframe[~dataframe["StockCode"].str.contains(r"^[A-Za-z]+", na=False)]    # POST and Manuel excluded but others stayed like 16168M 
    dataframe = dataframe[dataframe["Quantity"] > 0]
    dataframe = dataframe[dataframe["Price"] > 0]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    return dataframe

There is 38 different country in the dataset. Every country has own stracture. We can segment these countries with their characteristics or choose one them. Because when you give a transactional data to the apriori algorithm, that transforms to the matrix form and calculates the relationship between the products and a lot of data slows down the process.


* We are going to choose the country in this project


In [ ]:
df["Country"].value_counts().count()

38

In [ ]:
Country = input("Select a country for apriori").capitalize()
df_Country = df[df["Country"] == Country]

Select a country for aprioriGermany


When we looked our "summary_data" function, saw outlier, negative values in the numeric variables and some anomalies in the invoice and stock code variables. 

1. We know with our business information if invoice variable contains "C" in the begining of the variable, it means the product has been returned. In this point, we have to remove this products from the invoice. We try to more generalizable this stage and removed all of the letter contain products.

2. We know that stock code can contain letters but not all letters. Then again for being generalizable, we used regular expression to exclude this type of stock codes like POST and Manuel.

3. Quantity and Price variables cant contain negative values, these negative values removed.

4. When we look at the Quantity and Price variables quantiles, we can see extreme values are exist. These extreme values excluded from the dataset. 

In [ ]:
summary_data(df_Country)

  Head   :
     Invoice StockCode                          Description  Quantity         InvoiceDate  Price  Customer ID  Country
1109  536527     22809              SET OF 6 T-LIGHTS SANTA         6 2010-12-01 13:04:00   2.95      12662.0  Germany
1110  536527     84347  ROTATING SILVER ANGELS T-LIGHT HLDR         6 2010-12-01 13:04:00   2.55      12662.0  Germany
1111  536527     84945   MULTI COLOUR SILVER T-LIGHT HOLDER        12 2010-12-01 13:04:00   0.85      12662.0  Germany
1112  536527     22242        5 HOOK HANGER MAGIC TOADSTOOL        12 2010-12-01 13:04:00   1.65      12662.0  Germany
1113  536527     22244           3 HOOK HANGER MAGIC GARDEN        12 2010-12-01 13:04:00   1.95      12662.0  Germany
  Shape   :
(9495, 8)
  Describe   :
                0.00      0.01      0.05      0.10      0.25      0.50      0.75      0.95     0.99     1.00
Quantity      -288.0     -4.00      1.00      2.00      5.00     10.00     12.00     32.00     75.0    600.0
Price            0.0

After the cleaning, we lost more than 800 data but most of them because of the returned products.

In [ ]:
df_Country = retail_data_prep(df_Country)
summary_data(df_Country)

  Head   :
     Invoice StockCode                          Description  Quantity         InvoiceDate  Price  Customer ID  Country
1109  536527     22809              SET OF 6 T-LIGHTS SANTA       6.0 2010-12-01 13:04:00   2.95      12662.0  Germany
1110  536527     84347  ROTATING SILVER ANGELS T-LIGHT HLDR       6.0 2010-12-01 13:04:00   2.55      12662.0  Germany
1111  536527     84945   MULTI COLOUR SILVER T-LIGHT HOLDER      12.0 2010-12-01 13:04:00   0.85      12662.0  Germany
1112  536527     22242        5 HOOK HANGER MAGIC TOADSTOOL      12.0 2010-12-01 13:04:00   1.65      12662.0  Germany
1113  536527     22244           3 HOOK HANGER MAGIC GARDEN      12.0 2010-12-01 13:04:00   1.95      12662.0  Germany
  Shape   :
(8658, 8)
  Describe   :
                 0.00      0.01      0.05      0.10      0.25      0.50      0.75     0.95      0.99      1.00
Quantity         1.00      1.00      2.00      3.00      6.00     12.00     12.00     36.0     96.00    238.50
Price           

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In the cleaning section, we wanted to clean some stock codes and used regular expression. I wanted to control in here, is it still contain some of the stock codes that include letter? Yes, we continue..


In [ ]:
df_Country[df_Country['StockCode'] == "85123A"]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
497798,578472,85123A,WHITE HANGING HEART T-LIGHT HOLDER,12.0,2011-11-24 12:40:00,2.95,12476.0,Germany


At this point, we will choose 3 stock codes and recommend products to people who will buy these products.

In [ ]:
User1 = 21987
User2 = 23235
User3 = 22747

CheckID function gives us a description when we write the stock code.


In [ ]:
def CheckID(data, stock_code):
    product_name = data[data["StockCode"] == stock_code][["Description"]].values[0].tolist()
    print(product_name)

CheckID(df_Country, User1)

['PACK OF 6 SKULL PAPER CUPS']


In [ ]:
CheckID(df_Country, User2)

['STORAGE TIN VINTAGE LEAF']


In [ ]:
CheckID(df_Country, User3)

["POPPY'S PLAYHOUSE BATHROOM"]


## Preparation of Association Rules

We can create an apriori algorithm for a stock code or product descripton. Before the aprioriRecommendation function, is created an appropriate table based on this selection. 

Need to transform the data as a one hot dataframe




In [ ]:
def invoiceProduct(data,product=True):
    if product:
        return data.pivot_table(index="Invoice",columns="StockCode",values="Quantity").fillna(0).applymap(lambda x: 1 if x > 0 else 0)
    else:
        return data.pivot_table(index="Invoice",columns="Description",values="Quantity").fillna(0).applymap(lambda x: 1 if x > 0 else 0)


Apriori Algorithm has a min_support parameter which is a ratio of the item occur and total transaction number. This parameter's default value is 0.5 which is too much for some items then we change this value to 0.01.



In [ ]:
def rules(data):
    frequent_itemsets = apriori(data, min_support=0.01, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
    return rules

Apriori Algorithm returns a table with Antecedents, Consequents, Antecedent_Support, Consequents_Support, Support, Confidence, Lift, Leverage and Conviction metrics. These metrics:

* Antecedents: An item (for this dataset is product) found in data. 
* Consequents: An item found in combination with the antecedent.
* Antecedent_Support: Probability of antecedent's occurrence. 
* Consequents_Support: Probability of consequent's occurrence. 
* Support: Probability of antecedent and consequent's together occurrence.
* Confidence: Probability of the items occurring together by the probability of antecedent's occurence. 
<ul type = "disc"><li> (Support / Antecedent Support) </li></ul>
* Lift: Proportion of the items occurring together and their expected probabilities. 
<ul type = "disc"><li> (Support/(Antecedent_Support*Consequents_Support)) </li></ul> This shows us that when the antecedent item is purchased, the probability of purchasing the   consequent item increases by lift.

* Leverage: Correlation between item sets by comparing the support of item sets under independence assumption. 
<ul type = "disc"><li> (Support-(Antecedent_Support*Consequents_Support)) </li></ul>
 Because of the difference in 
the formula, leverage favors item sets with higher support, while lift can find strong associations among less frequent item sets.
* Conviction: Antecedent item's expected value without consequent item. 
<ul type = "disc"><li> ((1-Support) / (1-Confidence))</li></ul>







In [ ]:
rules(invoiceProduct(df_Country, product=True)).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(16237),(22326),0.011287,0.252822,0.011287,1.000000,3.955357,0.008433,inf
1,(22326),(16237),0.252822,0.011287,0.011287,0.044643,3.955357,0.008433,1.034915
2,(20674),(20675),0.022573,0.033860,0.013544,0.600000,17.720000,0.012780,2.415350
3,(20675),(20674),0.033860,0.022573,0.013544,0.400000,17.720000,0.012780,1.629044
4,(20674),(20676),0.022573,0.038375,0.011287,0.500000,13.029412,0.010420,1.923251


## Suggestion a product to the buyers.

In the recommendation stage, we can manage different strategies by metrics. Generally, i like the lift metric which also gives items with low frequencies. 

We can filter before the recommendation using metrics by support, lift, confidence etc

### Which product you recommed who buys a product 21987? 
**21987 = PACK OF 6 SKULL PAPER CUPS**


In [ ]:
User = User1

The id of the most probability recommendation is **85099B**


In [ ]:
def aprioriRecommendation(data,count=1):
    df_Country_inv_pro = invoiceProduct(df_Country, product=True)
    Rules = rules(df_Country_inv_pro)
    sorted_rules = Rules.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in sorted_rules["antecedents"].items():
        for j in list(product):
            if j == User:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"]))
    recommendation_list = list({item for item_list in recommendation_list for item in item_list})
    return recommendation_list[:count]

aprioriRecommendation(rules)

['85099B']

In [ ]:
for i in aprioriRecommendation(rules):
    CheckID(df_Country, i)

['JUMBO BAG RED RETROSPOT']


The first 3 id of the most probability recommendations are **85099B, 22029, 20750**

With these functions, we can choose how many products we want to recommend and see what the selections descriptions are.

In [ ]:
aprioriRecommendation(rules,count=3)

['85099B', 22029, 20750]

In [ ]:
for i in aprioriRecommendation(rules,count = 3):
    CheckID(df_Country, i)

['JUMBO BAG RED RETROSPOT']
['SPACEBOY BIRTHDAY CARD']
['RED RETROSPOT MINI CASES']
